In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hk7797/stock-market-india")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\KIIT\.cache\kagglehub\datasets\hk7797\stock-market-india\versions\6


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [11]:
import os

# Path to your dataset directory
data_dir = r"C:\Users\KIIT\.cache\kagglehub\datasets\hk7797\stock-market-india\versions\6"
print("Files in dataset directory:", os.listdir(data_dir))



Files in dataset directory: ['FullData.h5', 'FullDataCsv', 'StockMarketHelper']


In [13]:
import pandas as pd

# Read HDF5 file using pandas
h5_file_path = os.path.join(data_dir, "FullData.h5")
df = pd.read_hdf(h5_file_path)

# Display the first few rows of the dataset
print(df.head())


ImportError: Missing optional dependency 'pytables'.  Use pip or conda to install pytables.

In [10]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# Step 1: Load the dataset
data_dir = r"C:\Users\KIIT\.cache\kagglehub\datasets\hk7797\stock-market-india\versions\6"
file_list = os.listdir(data_dir)
print("Files in dataset directory:", file_list)

# Ensure you load the correct CSV file
data_path = os.path.join(data_dir, "NSE-TATAGLOBAL.csv")  # Replace with your actual file name
df = pd.read_csv(data_path)
print(df.head())

# Step 2: Preprocess the data
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df = df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()

# Scale the data for LSTM
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))

# Step 3: Prepare data for Linear Regression
df['Close_shifted'] = df['Close'].shift(-1)
df.dropna(inplace=True)
X = df[['Open', 'High', 'Low', 'Close', 'Volume']].values
y = df['Close_shifted'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

# Evaluation for Linear Regression
print("Linear Regression Mean Absolute Error:", mean_absolute_error(y_test, lr_predictions))
print("Linear Regression Mean Squared Error:", mean_squared_error(y_test, lr_predictions))

# Step 4: Prepare data for LSTM
sequence_length = 60
X_lstm = []
y_lstm = []

for i in range(sequence_length, len(scaled_data)):
    X_lstm.append(scaled_data[i-sequence_length:i, 0])
    y_lstm.append(scaled_data[i, 0])

X_lstm = np.array(X_lstm)
y_lstm = np.array(y_lstm)
X_lstm = np.reshape(X_lstm, (X_lstm.shape[0], X_lstm.shape[1], 1))

# Split the data into training and testing sets for LSTM
split = int(0.8 * len(X_lstm))
X_train_lstm, X_test_lstm = X_lstm[:split], X_lstm[split:]
y_train_lstm, y_test_lstm = y_lstm[:split], y_lstm[split:]

# Step 5: Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train_lstm, y_train_lstm, epochs=20, batch_size=32)

# Step 6: Evaluate the LSTM model
predictions = model.predict(X_test_lstm)
predictions = scaler.inverse_transform(predictions.reshape(-1, 1))

# Step 7: Plot predictions vs actual values
plt.figure(figsize=(12, 6))
plt.plot(df.index[-len(y_test_lstm):], scaler.inverse_transform(y_test_lstm.reshape(-1, 1)), color='blue', label='Actual Prices')
plt.plot(df.index[-len(predictions):], predictions, color='red', label='Predicted Prices')
plt.title('Stock Price Prediction using LSTM')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()


Files in dataset directory: ['FullData.h5', 'FullDataCsv', 'StockMarketHelper']


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\KIIT\\.cache\\kagglehub\\datasets\\hk7797\\stock-market-india\\versions\\6\\NSE-TATAGLOBAL.csv'